# Automated segmentation of thorax CT scans using neural networks

### CS 180 MP 2
### Rianna Patricia Cruz

## Introduction 
The identification of tumors and abnormal masses in medical imaging scans is one of the most important steps in the treatment of such abnormal conditions. Such a essential step in the treatment process is often tedious and time-consuming, as it requires a medical professional to manually identify such masses igiven some medical images such as CT or MRI scans. Though such identification process might be straightforward for a medical professional, the time it takes for such scans to be manually and individually processed and analyzed by a medical professional causes delays which might be critical to a patient's treatment.

The objective of this study was to automatically segment four organs-at-risk (OAR): the heart, the aorta, the trachea, and the esophagus


In [1]:
import datasets

In [2]:
# print("[INFO] loading scan attributes...")
# ave_depth = get_ave_depth()
# memoize: 186.
ave_depth = 186
scans, masks = datasets.load_scans(3, ave_depth)
print("done loading data")

01
load_scans: loaded file
load_scans: loaded data
load_scans: computing scale to uniformity
load_scans: rescaling to uniformity
load_scans: final output scale is
(512, 512, 186)
(512, 512, 186)
02
load_scans: loaded file
load_scans: loaded data
load_scans: computing scale to uniformity
load_scans: rescaling to uniformity
load_scans: final output scale is
(512, 512, 186)
(512, 512, 186)
03
load_scans: loaded file
load_scans: loaded data
load_scans: computing scale to uniformity
load_scans: rescaling to uniformity
load_scans: final output scale is
(512, 512, 186)
(512, 512, 186)
done loading data


In [3]:
# transpose for the sake of splitting training and test data
scans = scans.T
masks = masks.T
scans.shape
masks.shape

(186, 512, 512, 3)

In [4]:
from sklearn.model_selection import train_test_split
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
print("main: splitting data...")
split = train_test_split(scans, masks, test_size=0.25, random_state=42)
(X_train, X_test, y_train, y_test) = split
print("main: done split")

main: splitting data...
main: done split


In [5]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X_train.shape[1:])
print(y_train.shape[1:])
print(X_test.shape[1:])
print(y_test.shape[1:])

(139, 512, 512, 3)
(139, 512, 512, 3)
(47, 512, 512, 3)
(47, 512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)
(512, 512, 3)


In [6]:
# transpose for the sake of splitting training and test data
X_train = X_train.T
y_train = y_train.T

X_test = X_test.T
y_test = y_test.T
X_train.shape
X_test.shape

(3, 512, 512, 47)

In [21]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(X_train.shape[1:])
print(y_train.shape[1:])
print(X_test.shape[1:])
print(y_test.shape[1:])

(3, 512, 512, 139)
(3, 512, 512, 139)
(3, 512, 512, 47)
(3, 512, 512, 47)
(512, 512, 139)
(512, 512, 139)
(512, 512, 47)
(512, 512, 47)


In [32]:
from keras.models import Sequential
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM
from keras.layers import Dense, Conv3D, Conv2D, Conv1D, Flatten, MaxPooling2D

In [33]:
# Training parameters
batch_size = 100 # original = 32
num_classes = 10
epochs = 30

In [42]:
print("main: creating model")
model = Sequential()
model.summary()

main: creating model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [43]:
model.add(Conv2D(139, kernel_size = 1, input_shape=(512,512, None)))
model.summary()

ValueError: The channel dimension of the inputs should be defined. Found `None`.

In [44]:

print("main: compiling model")
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

main: compiling model


TypeError: Sequential model cannot be built: model is empty. Add some layers first.

In [41]:
print("main: fitting model")
history = model.fit(X_train, y_train, 
          batch_size = batch_size,
          epochs=epochs, 
          verbose=1, 
          validation_data=(X_test,y_test))

main: fitting model


ValueError: Error when checking input: expected conv2d_8_input to have shape (512, 512, 139) but got array with shape (512, 512, 47)